 # How to use the BRON arangodb

In [1]:
# Change working directory
import os
cwd = os.path.split(os.getcwd())
if cwd[-1] == 'graph_db':
    os.chdir('..')

assert os.path.split(os.getcwd())[-1] == 'BRON'

In [2]:
import arango
from graph_db.bron_example_for_detecting_abuse_of_authentication_mechanism_report import main, REPORT_URL, MDR_URL, FIRE_EYE_URL

SERVER_IP = 'bron.alfa.csail.mit.edu'
USERNAME = 'guest'
PASSWORD = 'guest'
DB = 'BRON'

## Example connect to BRON arangodb and perform AQL query

In [3]:
query = """
FOR c IN tactic
    FILTER c.original_id == "TA0006"
    RETURN c
"""
client = arango.ArangoClient(hosts=f"http://{SERVER_IP}:8529")
db = client.db(DB, username=USERNAME, password=PASSWORD, auth_method="basic")
assert db.aql.validate(query)
cursor = db.aql.execute(query)
for c in cursor:
    print(c)


{'_key': 'tactic_00009', '_id': 'tactic/tactic_00009', '_rev': '_bmbn28S---', 'original_id': 'TA0006', 'datatype': 'tactic', 'name': 'credential-access', 'metadata': {}}


In [4]:
query_template = """
FOR c IN capec
    FILTER c.original_id == "{}"
    FOR v IN 1..1 ANY c {}
        RETURN v
"""
capec = "117"
edge_collections = ("TechniqueCapec", "CapecCwe")
for edge_collection in edge_collections:
    query = query_template.format(capec, edge_collection)
    assert db.aql.validate(query)
    cursor = db.aql.execute(query)
    results = {_['original_id'] for _ in cursor}
    print(f'{len(results)} result for CAPEC {capec} in {edge_collection}: {results}')


1 result for CAPEC 117 in TechniqueCapec: {'T1020.001'}
1 result for CAPEC 117 in CapecCwe: {'319'}


## Example report download, threat data extraction and BRON query

In [5]:
url = REPORT_URL
_ = main(SERVER_IP, PASSWORD, USERNAME, url)

Query results records: {'tactic': {'TA0006': defaultdict(<class 'int'>, {'technique': 49})}, 'technique': {'T1114.002': defaultdict(<class 'int'>, {'tactic': 1}), 'T1114': defaultdict(<class 'int'>, {'tactic': 1}), 'T1199': defaultdict(<class 'int'>, {'tactic': 1}), 'T1552': defaultdict(<class 'int'>, {'tactic': 1}), 'T1552.004': defaultdict(<class 'int'>, {'tactic': 1})}, 'cve': {}}
Query results number of traversals: {'tactic': 6, 'technique': 9, 'cve': 3}


## Example using file with a network description

In [6]:
network_description_file = 'graph_db/example_data/network_file_bron.json'
_ = main(SERVER_IP, PASSWORD, USERNAME, url, network_description_file=network_description_file)

Query results records: {'tactic': {'TA0006': defaultdict(<class 'int'>, {'technique': 49})}, 'technique': {'T1114.002': defaultdict(<class 'int'>, {'tactic': 1}), 'T1114': defaultdict(<class 'int'>, {'tactic': 1}), 'T1199': defaultdict(<class 'int'>, {'tactic': 1}), 'T1552': defaultdict(<class 'int'>, {'tactic': 1}), 'T1552.004': defaultdict(<class 'int'>, {'tactic': 1})}, 'cve': {}}
Query results number of traversals: {'tactic': 6, 'technique': 9, 'cve': 3}
Number of configurations in CPE format in network 18
Network matches: defaultdict(<class 'set'>, {'tactic': {'TA0006'}})


In [7]:
# TODO

# Link to network


# Using web interface instructions